In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms.functional as F
import os


In [ ]:
from torch.utils.data import DataLoader


In [ ]:
from data.dataloader import CustomDataset
from data.utils import DataTransform
from model.blocks.contourlet import ContourletTransform
from model.blocks.subbandfusion import SubbandFusionNet


In [ ]:
transform=DataTransform(image_size=640)

dataset = CustomDataset(
    path="data/1_train/1_LOLdataset",
    transform=transform
)


In [ ]:
dataloader = DataLoader(
    dataset=dataset,
    batch_size=16,
    shuffle=True,
    num_workers=int(os.cpu_count() * 0.9),
    persistent_workers=True,
    pin_memory=True
)


In [ ]:
data = next(iter(dataloader))
print(data.shape)


In [ ]:
def show_batch(images, ncols=4):
    nimgs = images.shape[0]
    nrows = (nimgs + ncols - 1) // ncols
    plt.figure(figsize=(ncols * 3, nrows * 3))
    for i in range(nimgs):
        plt.subplot(nrows, ncols, i + 1)
        plt.imshow(F.to_pil_image(images[i]))
        plt.axis('off')
        plt.title(f"Image {i}")
    plt.tight_layout()
    plt.show()

show_batch(images=data, ncols=8)


In [ ]:
contourlet = ContourletTransform(
    num_levels=4,
    filter_size=5,
    sigma=1.0,
    omega_x=0.25,
    omega_y=0.25,
    channels=3
)

pyramid, subbands = contourlet(data)
show_batch(images=pyramid[-1], ncols=8)


In [ ]:
subbandfusion = SubbandFusionNet(
    in_channels=3,
    base_channels=64,
    num_levels=4,
    num_heads=8
)

fusion = subbandfusion(subbands)

show_batch(images=fusion, ncols=8)
